# Seperate File for Multiprocessing

In [ ]:
import pandas as pd #data analysis
# import os #file/directory operations
import multiprocessing as mp
import DC_Pickle as dcp
# from six.moves import cPickle as pickle

datafile='../../data/v2merged_test.csv' #full dataset
df=pd.read_csv(datafile)
print("\n - importing data... ")
        
groups = df.groupby('ga:eventAction')
print("\n - Data has {0} individualas with {1} data".format(len(groups), len(df)))

cores = mp.cpu_count() # number of cores

dcp.make_folders("../../data/pickles/multiprocessing_test/") # grouping data based on core number
print("\n - Seperate data into {0} groups including individuals".format(cores, round(len(groups)/cores)))

dcp.seperate_indi(groups, cores, "../../data/pickles/multiprocessing_test/process{0}.pickle")

# sleepgap.py를 실행해 result_sleepgap_15.csv 파일 생성

In [11]:
import numpy as np
import pandas as pd

datafile='../../data/v2merged_test.csv' #full dataset
df=pd.read_csv(datafile)
shor_gap_length=15 #analysis extremely robust to changes in this parameter

#shor_gap_length1 = pd.DataFrame(np.ones, columns='aa')

shor_gap_length1 = pd.DataFrame({'shor_gap':np.ones(len(df))*shor_gap_length})
print(shor_gap_length1)

       shor_gap
0            15
1            15
2            15
3            15
4            15
5            15
6            15
7            15
8            15
9            15
10           15
11           15
12           15
13           15
14           15
15           15
16           15
17           15
18           15
19           15
20           15
21           15
22           15
23           15
24           15
25           15
26           15
27           15
28           15
29           15
...         ...
13805        15
13806        15
13807        15
13808        15
13809        15
13810        15
13811        15
13812        15
13813        15
13814        15
13815        15
13816        15
13817        15
13818        15
13819        15
13820        15
13821        15
13822        15
13823        15
13824        15
13825        15
13826        15
13827        15
13828        15
13829        15
13830        15
13831        15
13832        15
13833        15
13834        15

[13835 

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd #data analysis
import time
import multiprocessing as mp
from multiprocessing import Process, Queue
import DC_Pickle as dcp
import DC_filter as dcf
t0 = time.clock() # initial time
cores= mp.cpu_count() #processes raw data in batches (required because data>ram)
loding = True

def process_df(queue, file_name):
    err_players = []
    
    df = dcp.open_Pickle(file_name)
    df = df.drop('Unnamed: 0', axis=1)
    
    new_df = df.groupby('ga:eventAction').apply(dcf.add_TotalPlays)
    new_df = new_df[new_df['total_plays']>14]
    new_df = new_df[new_df['ga:eventLabel']<301]
    dcf.check_Discon(new_df, err_players)
                  
    new_df = dcf.add_DiffTime(new_df)
    new_df = dcf.add_GapType(new_df)
    #df = filter_time(df) # 이건 안하는게 좋겠다. attemps를 지우니까.
    new_df=new_df.groupby('ga:eventAction').apply(dcf.add_GapCategory)
                  
    queue.put([new_df, err_players])

## Execute! ##
print('\n -  Data Loading...')

## start multiprocessing
for i in range(cores):
    set_filename = "../../data/pickles/multiprocessing_test/process{0}.pickle".format(i)
    set_pros_name = "core{0}".format(i)
    set_q_name = "q{0}".format(i)

    globals()[set_q_name] = Queue()
    globals()[set_pros_name] = Process(target=process_df, args=(eval(set_q_name), set_filename,))
    #globals()[set_pros_name] = Process(target=process_df, args=(eval(set_q_name), set_filename, eval(error_name),))

    eval(set_pros_name).start()
    print('core{0} start'.format(i))
    time.sleep(1)

print('\n -  Data filtering...')

## get returned value from multiprocessing
df = pd.DataFrame()
err = pd.DataFrame()
for i in range(cores):
    set_q_name = "q{0}".format(i) # queue name
    set_pros_name = "core{}".format(i) # process name
    set_df_name = "df{0}".format(i) # data frame name of each process
    error_name = "error_players{0}".format(i) # player name with error attempts
    
    locals()[set_df_name], locals()[error_name] = eval(set_q_name).get() # get from queue

    df = df.append(eval(set_df_name)) # concat each dfs
    err = err.append(eval(error_name)) # np.hstack((err_plys, eval(error_name))) # concat each error players
    #df = df.drop('level_0', axis=1)
    
    eval(set_q_name).close()
    eval(set_pros_name).join()
    print('core{0} close'.format(i))
    time.sleep(1)

[df, err] = dcf.data_PostProcess(df, err)

print("\n - saving processed data...\n {0} plays, {1}players".format(len(df), len(df.groupby('eventAction'))))

df.to_csv('../../data/filtered_data_test.csv')
err.to_csv('../../data/error_players_test.csv')
print("\n - process terminal.")
print("Run time: ", time.clock()-t0)

# Hash table style

import pandas as pd #data analysis
import numpy as np #numerical function
from six.moves import cPickle as pickle
df=pd.read_csv('filtered_data_test.csv')
print(df.columns)
df = df.drop('Unnamed: 0', axis=1)
df = df.drop('index', axis=1)
print(df.columns)

### make a matrix that will contain each column of each player

In [ ]:
import numpy as np #numerical function
import pandas as pd #data analysis
import os #file/directory operations
from six.moves import cPickle as pickle

df=pd.read_csv('filtered_data_test.csv')
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
def spt_ColData(data_frame, col_name): # seperate columns into each matrix
    '''
    def group_Len(data, len_vec): # inner function
        len_vec.append(len(data))

    group_len = []
    groups = df.groupby('ga:eventAction') # grouping
    groups.apply(group_Len, group_len) # get length of each player

    #print(max(group_len), len(groups)) # row = attemt numbers, column = players
    '''
    groups = df.groupby('eventAction') # grouping
    col_matrix = np.ones((301, len(groups)))*np.nan # make a matrix with player attempts size.
    
    #if type(group[col_name]) == str:
    #col_matrix = np.chararray((301, len(groups)), itemsize=37)
    #col_matrix = np.zeros((301, len(groups)))

    count = 0
    for name, group in groups:
        idx = np.array(group['eventLabel'])-1
        col_matrix[idx, count] = group[col_name]
        count = count + 1
    #print('Matrix size:', np.shape(col_matrix))
    return col_matrix

def make_folders(set_dir_name):
    if os.path.exists(set_dir_name):
      # You may override by setting force=True.
      print('%s folder already present - Skipping pickling.' % set_dir_name)
    else:
        try:
            os.makedirs(set_dir_name)
        except Exception as e:
            print('Unable to make', set_dir_name, ':', e)
            return

def make_Pickle(set_data, set_filename, force = False):
    if os.path.exists(set_filename) and not force:
        # You may override by setting force=True.  
        print('%s pickle already present - Skipping pickling.' % set_filename)
    else :
        try :
            with open(set_filename, 'wb') as f:
                pickle.dump(set_data, f, pickle.HIGHEST_PROTOCOL)
        except Exception as e :
            print("Unable to save data to", set_filename, ': ', e)

In [ ]:
# print(df.columns)
mm = spt_ColData(df, 'eventLabel')
#mm = mm[:15, :]
print(mm[:, 0])

#### conver to picke except 'eventAction' and 'comb_time'

In [ ]:
def colToPickle():
    set_folder_name = 'pickles/seperate_origin/'
    make_folders(set_folder_name)    
    
    for col_name in np.array(df.columns):
        if not (col_name == 'eventAction' or col_name == 'comb_time'): # conver to picke except 'eventAction' and 'comb_time'
            set_mat = col_name
            print(col_name)
            locals()[set_mat] = spt_ColData(df, col_name)
            make_Pickle(eval(set_mat), 'pickles/seperate_origin/{0}'.format(col_name))

In [ ]:
colToPickle()

groups = df.groupby('eventAction')
names = pd.DataFrame({'ID':groups.grouper.result_index.values})
make_Pickle(names, 'pickles/seperate_origin/eventAction')